In [1]:
from typing import TYPE_CHECKING


if TYPE_CHECKING:
    from math_rag.application.containers import ApplicationContainer
    from math_rag.infrastructure.containers import InfrastructureContainer

    application_container: ApplicationContainer
    infrastructure_container: InfrastructureContainer

In [2]:
RESET = False
%load_ext hooks.notebook_hook

2025-07-13 10:02:39,802 - INFO - datasets - config.py:54 - PyTorch version 2.6.0 available.


## Setup

In [4]:
math_expression_description_writer_assistant = (
    application_container.math_expression_description_writer_assistant()
)
math_expression_description_optimizer_assistant = (
    application_container.math_expression_description_optimizer_assistant()
)
math_expression_comparator_assistant = application_container.math_expression_comparator_assistant()
math_expression_relationship_description_writer_assistant = (
    application_container.math_expression_relationship_description_writer_assistant()
)
math_expression_relationship_detector_assistant = (
    application_container.math_expression_relationship_detector_assistant()
)

default_embedder = application_container.default_embedder()
math_expression_description_opt_embedding_repository = (
    infrastructure_container.math_expression_description_opt_embedding_repository()
)
math_expression_description_opt_repository = (
    infrastructure_container.math_expression_description_opt_repository()
)
math_expression_description_repository = (
    infrastructure_container.math_expression_description_repository()
)
math_expression_group_repository = infrastructure_container.math_expression_group_repository()
math_expression_repository = infrastructure_container.math_expression_repository()
grouper_service = application_container.grouper_service()

math_expression_graph_repository = await infrastructure_container.math_expression_graph_repository()
math_expression_relationship_repository = (
    infrastructure_container.math_expression_relationship_repository()
)
math_expression_relationship_description_repository = (
    infrastructure_container.math_expression_relationship_description_repository()
)
katex_corrector_service = application_container.katex_corrector_service()
math_expression_context_repository = infrastructure_container.math_expression_context_repository()
math_article_chunk_repository = infrastructure_container.math_article_chunk_repository()
math_expression_index_repository = infrastructure_container.math_expression_index_repository()

In [4]:
from pathlib import Path

from math_rag.core.models import MathArticle


google_drive_repository = infrastructure_container.google_drive_repository()
math_article_parser_service = infrastructure_container.math_article_parser_service()

file_id = google_drive_repository.get_file_id(
    Path('ml/lectures/L07-LogisticRegression2/2024_08_10_2174b40686820b4cb591g.tex')
)

if not file_id:
    raise ValueError()

file_content = google_drive_repository.get_file_by_id(file_id)

math_article = MathArticle(
    math_expression_dataset_id=None,
    math_expression_index_id=None,
    name='article',
    bytes=file_content.getvalue(),
)

2025-07-13 00:23:32,301 - INFO - googleapiclient.discovery_cache - __init__.py:49 - file_cache is only supported with oauth2client<4.0.0


In [ ]:
from uuid import UUID

from math_rag.application.utils import InputCreatorUtil
from math_rag.core.models import MathExpressionIndex
from math_rag.infrastructure.utils import (
    TemplateChunkerUtil,
    TemplateContextChunkerUtil,
    TemplateFormatterUtil,
    TemplateIndexFinderUtil,
)


index = MathExpressionIndex()
index.id

In [5]:
index = MathExpressionIndex(id=UUID('6cafc7fa-982b-4285-92d0-a53dd2a8453a'))
index.id

UUID('6cafc7fa-982b-4285-92d0-a53dd2a8453a')

In [ ]:
# TODO
# - dont chunk over image placeholder (use code like in context chunker)
# rename payload "description" in embeddings

In [30]:
index_id_to_remove = UUID('01627335-ba50-4c7d-85f2-eb0b27027c6f')
common_filter = dict(math_expression_index_id=index_id_to_remove)

await math_expression_index_repository.delete_one(filter=dict(id=index_id_to_remove))

await math_expression_repository.delete_many(filter=common_filter.copy())
await math_expression_context_repository.delete_many(filter=common_filter.copy())
await math_expression_description_repository.delete_many(filter=common_filter.copy())
await math_expression_description_opt_repository.delete_many(filter=common_filter.copy())
await math_expression_group_repository.delete_many(filter=common_filter.copy())
await math_article_chunk_repository.delete_many(filter=common_filter.copy())
await math_expression_relationship_repository.delete_many(filter=common_filter.copy())
await math_expression_relationship_description_repository.delete_many(filter=common_filter.copy())
await math_expression_description_opt_embedding_repository.clear()

## Nodes

### 1. MathExpression, requires: MathArticle

In [7]:
from math_rag.core.models import MathExpression


math_nodes, _, template = math_article_parser_service.parse_for_index(math_article)
math_nodes.sort(key=lambda x: x.position)

katexes = [math_node.latex.strip('$') for math_node in math_nodes]
valid_katexes = await katex_corrector_service.correct(katexes, max_num_retries=3)
math_expressions = [
    MathExpression(
        math_article_id=math_article.id,
        math_expression_dataset_id=None,
        math_expression_group_id=None,
        math_expression_index_id=index.id,
        latex=node.latex,
        katex=katex.strip(),
        index=i,
        position=node.position,
        is_inline=node.is_inline,
    )
    for i, (node, katex) in enumerate(zip(math_nodes, valid_katexes))
]
await math_expression_repository.insert_many(math_expressions)

In [8]:
print(template)


Jan Šnajder, lectures, v2.0

Last time we introduced the logistic regression algorithm. We defined the model and derived the cross-entropy error function as the negative probability of the labels in the training set. We established that minimizing that error had no solution in closed form, so we turned to iterative procedures. We have considered the simplest such procedure, the gradient descent algorithm, and we applied it to logistic regression, in standard (batch) and stochastic variant. In the end, we talked about regularization, specifically [math_placeholder | 0] regularization, which we incorporated quite straightforwardly into the optimization process.

Today we'll talk a bit more about logistic regression. First, we'll consider some more efficient (read: faster) alternatives to gradient descent. Second, we'll consider the extension of binary logistic regression to multiclass logistic regression. Third, we'll look at all the models discussed thus far and see what they have in c

### 2. MathExpressionContext, requires MathExpression

In [9]:
context_templates = TemplateContextChunkerUtil.chunk(template, max_context_size=1000)
assert len(context_templates) == len(math_expressions)

print(context_templates[1])

above consideration, we can conclude that the batch gradient descent could be improved if we take into account not only the slope (gradient) but also the curvature (the change in gradient, i.e., the second derivative) of the error function. Such optimization methods are referred to as second-order optimization, as opposed to first-order optimization methods, such as gradient descent. The basic second-order optimization method is the Newton's method.


Consider minimization of function [math_placeholder | 1]. We know that the parameter update in gradient descent is as follows:

[math_placeholder | 2]

If we introduce an index for the iterations, then we can write this as an equation:

[math_placeholder | 3]

The idea with Newton's method is to take the point [math_placeholder | 4] (the current minimum) and compute at it the quadratic approximation of the function [math_placeholder | 5], and then move to the minimizer of this quadratic approximation (which is known analytically). If [mat

In [10]:
from math_rag.core.models import MathExpressionContext


index_to_katex = {
    math_expression.index: math_expression.katex for math_expression in math_expressions
}
math_expression_contexts: list[MathExpressionContext] = []

for math_expression, context_template in zip(math_expressions, context_templates):
    formatted_context, _ = TemplateFormatterUtil.format(
        context_template, index_to_katex, omit_wrapper=False
    )
    math_expression_context = MathExpressionContext(
        math_article_id=math_article.id,
        math_expression_id=math_expression.id,
        math_expression_index_id=index.id,
        text=formatted_context,
    )
    math_expression_contexts.append(math_expression_context)

await math_expression_context_repository.insert_many(math_expression_contexts)

### 3. MathExpressionDescription, requires MathExpression, MathExpressionContext

In [11]:
from math_rag.application.models.assistants.inputs import (
    MathExpressionDescriptionWriter as AssistantInput,
)
from math_rag.core.models import MathExpressionDescription
from math_rag.infrastructure.constants.services import MATH_TEMPLATE


inputs: list[AssistantInput] = []
input_id_to_math_expression_id: dict[UUID, UUID] = {}

for math_expression, math_expression_context in zip(math_expressions, math_expression_contexts):
    input = AssistantInput(
        katex=MATH_TEMPLATE.format(katex=math_expression.katex, index=math_expression.index),
        context=math_expression_context.text,
    )
    inputs.append(input)
    input_id_to_math_expression_id[input.id] = math_expression.id

outputs = await math_expression_description_writer_assistant.concurrent_assist(inputs)
math_expression_descriptions = [
    MathExpressionDescription(
        math_expression_index_id=index.id,
        math_expression_id=input_id_to_math_expression_id[output.input_id],
        text=output.description,
    )
    for output in outputs
]
await math_expression_description_repository.insert_many(math_expression_descriptions)

In [12]:
for x in math_expression_descriptions:
    print(x.id)
    print(x.text)
    print('-----')
    print()

0f273a6e-37ce-482e-9d30-60e722f61113
The number of classes in a classification problem is greater than two.
-----

688ca751-615d-4e3c-93c5-aff8fffb7fbf
The error function expressed as a function of the vector variable w, conditioned on the dataset D.
-----

d9873aaa-b2f3-4436-8800-d61b2bb00329
the dimension of the output vector produced by the softmax function, which matches the number of classes
-----

db1c0de7-de27-4af7-906a-344e35354250
the dimensionality of the vector produced by the softmax function, corresponding to the number of classes
-----

afa448f5-9adb-4b16-a288-5773e6707376
the point at which the quadratic approximation of the function is constructed and is tangential to the function during the minimization process
-----

20c561d7-0d6c-4c10-ba35-ad688ee04188
The variable representing the number of features in the feature space after mapping.
-----

73463273-b6ea-47ad-83ab-5727406a30c5
A general differentiable function of a single variable, referenced in the context of Tayl

### 4. MathExpressionDescriptionOpt, requires: MathExpressionDescription

In [13]:
from math_rag.application.models.assistants.inputs import (
    MathExpressionDescriptionOptimizer as AssistantInput,
)
from math_rag.core.models import MathExpressionDescriptionOpt


inputs, input_id_to_math_expression_description = InputCreatorUtil.create(
    math_expression_descriptions, lambda x: AssistantInput(description=x.text)
)
outputs = await math_expression_description_optimizer_assistant.concurrent_assist(inputs)
math_expression_descriptions_opt = [
    MathExpressionDescriptionOpt(
        math_expression_id=input_id_to_math_expression_description[
            output.input_id
        ].math_expression_id,
        math_expression_description_id=input_id_to_math_expression_description[output.input_id].id,
        math_expression_index_id=index.id,
        text=output.description,
    )
    for output in outputs
]

In [14]:
for x in math_expression_descriptions_opt:
    print(x.math_expression_description_id)
    print(x.text)
    print('-----')
    print()

e3c05985-b221-428e-bc2f-1ac4ff5360da
The norm of the parameter vector w approaches infinity.
-----

688ca751-615d-4e3c-93c5-aff8fffb7fbf
Error function as a function of vector variable w, conditioned on dataset D.
-----

20c561d7-0d6c-4c10-ba35-ad688ee04188
Variable denoting the number of features in the feature space after mapping.
-----

cf4956ac-d57e-4f59-bfe4-dc0d5cc867c6
Point in n-dimensional space where function f is evaluated and Hessian matrix is computed.
-----

73463273-b6ea-47ad-83ab-5727406a30c5
Differentiable single-variable function used in Taylor series expansion and quadratic approximation.
-----

4169742a-47bc-46dc-8c02-3699764eb94c
Variable representing a specific class label, used as an index to distinguish among multiple classes in a multinomial logistic regression model.
-----

598293f6-868c-424b-b239-246b789e6a0e
Mapping from n-dimensional real vectors to n-dimensional real vectors.
-----

f891481f-75ed-47af-9fde-63464ccafee4
Hessian matrix of a vector-valued fun

In [15]:
from more_itertools import unzip

from math_rag.application.models.embedders import EmbedderInput


inputs, input_id_to_item = InputCreatorUtil.create(
    math_expression_descriptions_opt, lambda x: EmbedderInput(text=x.text)
)
outputs = await default_embedder.concurrent_embed(inputs)
descriptions, embeddings = unzip(
    (input_id_to_item[output.input_id], output.embedding) for output in outputs
)
descriptions, embeddings = list(descriptions), list(embeddings)
await math_expression_description_opt_repository.insert_many(descriptions)
await math_expression_description_opt_embedding_repository.upsert_many(descriptions, embeddings)

### 5. MathExpressionGroup, requires: MathExpressionDescription, MathExpressionContext

In [8]:
grouped_descriptions = await math_expression_description_opt_embedding_repository.group(
    grouper_service.group
)

In [45]:
from qdrant_client.http.models import Record


grouped_records: list[list[Record]] = []

for descriptions in grouped_descriptions:
    ids = [x.id for x in descriptions]
    records = await math_expression_description_opt_embedding_repository.client.retrieve(
        collection_name=math_expression_description_opt_embedding_repository.collection_name,
        ids=[str(id) for id in ids],
        with_payload=True,
        with_vectors=True,
    )

    for record in records:
        # remove some data for a clener diagram
        record.payload['description'] = record.payload['description'][:50]
        record.payload.pop('math_expression_description_id')
        record.payload.pop('math_expression_index_id')
        record.payload.pop('timestamp')

    grouped_records.append(records)

In [ ]:
# grouped_records[0][0].payload

{'math_expression_id': '419e63ac-31f8-4c46-a058-0705a0b746f2',
 'math_expression_description_id': 'ff8ec75f-7c82-4d0f-a71d-296807e59033',
 'math_expression_index_id': '6cafc7fa-982b-4285-92d0-a53dd2a8453a',
 'timestamp': '2025-07-12T22:43:37.991873',
 'description': 'square matrix with n rows and n columns'}

In [30]:
import os

import pandas as pd
import plotly.express as px

from sklearn.datasets import make_blobs


os.environ['NUMBA_CPU_FEATURES'] = str()  # avoid kernel crash on arm
import umap

#### Example data

In [ ]:
# synthetic data
X, y = make_blobs(
    n_samples=500,
    centers=5,
    n_features=10,
    cluster_std=1.0,
    random_state=42,
)

reducer = umap.UMAP(
    n_components=2,
    metric='euclidean',
    random_state=None,
)
X_umap = reducer.fit_transform(X)

In [ ]:
df = pd.DataFrame(
    {
        'UMAP1': X_umap[:, 0],
        'UMAP2': X_umap[:, 1],
        'cluster': y,
    }
)

fig = px.scatter(
    df,
    x='UMAP1',
    y='UMAP2',
    color='cluster',
    hover_data=['cluster'],
)
fig.show()

#### Real data

In [ ]:
records = [r for grp in grouped_records for r in grp]
vectors = [r.vector for r in records]
cluster_labels = [i for i, grp in enumerate(grouped_records) for _ in grp]

# figure out which payload keys exist across all records
payload_keys = set().union(*(r.payload.keys() for r in records))

reducer = umap.UMAP(
    n_components=2,
    metric='cosine',
    random_state=None,
)
X_umap = reducer.fit_transform(vectors)

In [47]:
rows = []
for x, y, label, record in zip(X_umap[:, 0], X_umap[:, 1], cluster_labels, records):
    row = {
        'UMAP1': x,
        'UMAP2': y,
        'cluster': label,
        'id': record.id,
    }
    row.update(record.payload or {})  # add all payload fields
    rows.append(row)

df = pd.DataFrame(rows)

fig = px.scatter(
    df,
    x='UMAP1',
    y='UMAP2',
    color='cluster',
    hover_data=list(payload_keys) + ['id', 'cluster'],
)
fig.show()

In [9]:
from itertools import combinations

from math_rag.application.models.assistants.inputs import MathExpressionComparator as AssistantInput
from math_rag.application.utils import GroupPrunerUtil
from math_rag.core.models import MathExpressionGroup


grouped_math_expression_ids = [
    [description.math_expression_id for description in descriptions]
    for descriptions in grouped_descriptions
]

for math_expression_ids in grouped_math_expression_ids:
    math_expressions = await math_expression_repository.find_many(
        filter=dict(id=math_expression_ids)
    )
    math_expression_contexts = await math_expression_context_repository.find_many(
        filter=dict(math_expression_id=math_expression_ids)
    )
    pairs = list(combinations(zip(math_expressions, math_expression_contexts), 2))

    if not pairs:
        continue

    inputs: list[AssistantInput] = []
    input_id_to_candidate_pair: dict[UUID, tuple[UUID, UUID]] = {}

    for pair, other_pair in pairs:
        math_expression, math_expression_context = pair
        other_math_expression, other_math_expression_context = other_pair
        input = AssistantInput(
            katex=math_expression.katex,
            context=math_expression_context.text,
            other_katex=other_math_expression.katex,
            other_context=other_math_expression_context.text,
        )
        inputs.append(input)
        input_id_to_candidate_pair[input.id] = (math_expression.id, other_math_expression.id)

    outputs = await math_expression_comparator_assistant.concurrent_assist(inputs)

    candidates = math_expression_ids
    candidate_pair_to_is_connected = {
        input_id_to_candidate_pair[output.input_id]: output.is_identical for output in outputs
    }

    math_expression_ids_to_group = GroupPrunerUtil.prune(candidates, candidate_pair_to_is_connected)
    math_expression_group = MathExpressionGroup(math_expression_index_id=index.id)

    await math_expression_group_repository.insert_one(math_expression_group)
    await math_expression_repository.update_group_id(
        math_expression_ids_to_group, math_expression_group.id
    )

In [ ]:
# insert updated math expressions to the graph database
for math_expression_ids in grouped_math_expression_ids:
    math_expressions = await math_expression_repository.find_many(
        filter=dict(id=math_expression_ids)
    )
    await math_expression_graph_repository.insert_many_nodes(math_expressions)

## Relationships

### 1. MathArticleChunk, requires: MathExpression

In [7]:
math_expressions = await math_expression_repository.find_many(
    filter=dict(math_expression_index_id=index.id)
)

In [ ]:
from math_rag.core.models import MathArticleChunk


index_to_katex = {
    math_expression.index: math_expression.katex for math_expression in math_expressions
}
chunk_templates = TemplateChunkerUtil.chunk(template, max_window_size=2048, max_padding=256)
math_article_chunks: list[MathArticleChunk] = []

for i, chunk_template in enumerate(chunk_templates):
    indexes = TemplateIndexFinderUtil.find(chunk_template)

    if len(indexes) < 2:
        continue

    formatted_chunk, _ = TemplateFormatterUtil.format(
        chunk_template, index_to_katex, omit_wrapper=False
    )
    # print(_)
    math_article_chunk = MathArticleChunk(
        math_article_id=math_article.id,
        math_expression_index_id=index.id,
        index=i,
        indexes=indexes,
        text=formatted_chunk,
    )
    math_article_chunks.append(math_article_chunk)

await math_article_chunk_repository.insert_many(math_article_chunks)

### 2. MathExpressionRelationship, requires: MathExpression, MathArticleChunk

In [7]:
math_expressions = await math_expression_repository.find_many(
    filter=dict(math_expression_index_id=index.id)
)
math_article_chunks = await math_article_chunk_repository.find_many(
    filter=dict(math_expression_index_id=index.id)
)

In [ ]:
num_expected_chunks = sum(
    len(math_article_chunk.indexes) - 1 for math_article_chunk in math_article_chunks
)
num_expected_chunks

# TODO why did we get 2389 results?

2735

In [8]:
from math_rag.application.models.assistants.inputs import (
    MathExpressionRelationshipDetector as AssistantInput,
)
from math_rag.core.models import MathExpressionRelationship


for math_article_chunk in math_article_chunks:
    if len(math_article_chunk.indexes) < 2:
        continue

    start_indexes = math_article_chunk.indexes[:-1]
    last_index = math_article_chunk.indexes[-1]
    index_pairs = [(index, last_index) for index in start_indexes]
    inputs: list[AssistantInput] = []
    input_id_to_math_expression_id_pair: dict[UUID, tuple[UUID, UUID]] = {}
    input_id_to_math_expression_index_pair: dict[UUID, tuple[int, int]] = {}

    for source_index, target_index in index_pairs:
        input = AssistantInput(
            chunk=math_article_chunk.text, source=source_index, target=target_index
        )
        inputs.append(input)

        source_math_expression = next(
            (x for x in math_expressions if x.index == source_index), None
        )
        target_math_expression = next(
            (x for x in math_expressions if x.index == target_index), None
        )

        if source_math_expression is None or target_math_expression is None:
            raise ValueError()

        input_id_to_math_expression_id_pair[input.id] = (
            source_math_expression.id,
            target_math_expression.id,
        )
        input_id_to_math_expression_index_pair[input.id] = source_index, target_index

    outputs = await math_expression_relationship_detector_assistant.concurrent_assist(inputs)
    math_expression_relationships = [
        MathExpressionRelationship(
            math_article_chunk_id=math_article_chunk.id,
            math_expression_index_id=index.id,
            math_expression_source_id=input_id_to_math_expression_id_pair[output.input_id][0],
            math_expression_target_id=input_id_to_math_expression_id_pair[output.input_id][1],
            math_expression_source_index=input_id_to_math_expression_index_pair[output.input_id][0],
            math_expression_target_index=input_id_to_math_expression_index_pair[output.input_id][1],
        )
        for output in outputs
        if output.relationship_exists
    ]
    await math_expression_relationship_repository.insert_many(math_expression_relationships)

### 3. MathExpressionRelationshipDescription, requires: MathArticleChunk, MathExpressionRelationship

In [ ]:
math_expression_relationships = await math_expression_relationship_repository.find_many(
    filter=dict(math_expression_index_id=index.id)
)
len(math_expression_relationships)

2389

In [42]:
await math_article_chunk_repository.find_one(filter=dict(index=0))
# TODO missing math_article_chunk with index 0??

In [44]:
await math_article_chunk_repository.find_many(filter=dict(math_expression_index_id=[index.id]))

[MathArticleChunk(id=UUID('29995be7-5304-4c5e-bdb6-f78370a0bc8b'), math_article_id=UUID('03bd6e71-1c57-4a16-9f8d-86d7a3fc6412'), math_expression_index_id=UUID('6cafc7fa-982b-4285-92d0-a53dd2a8453a'), timestamp=datetime.datetime(2025, 7, 13, 0, 8, 17, 717000), index=142, indexes=[153, 154, 155, 156, 157, 158], text="generalized linear models are models that wrap the scalar product of the weight vector and example vector into an activation function [f | 153]. So:\n\n[h(\\mathbf{x} ; \\mathbf{w})=f\\left(\\mathbf{w}^{\\mathrm{T}} \\boldsymbol{\\phi}(\\mathbf{x})\\right) | 154]\n\nLet's look at three generalized linear models we've considered so far. For each of them, let's consider four points: (1) how the model is defined, (2) what is the probability distribution to which their output corresponds, (3) how is the loss is defined, and (4) what is the gradient of the loss, which we need for gradient descent.\n\nFirst, let's look at the linear regression algorithm:\n\n[\\begin{aligned}\nh(\\

In [53]:
await math_article_chunk_repository.find_many(
    filter=dict(id=[UUID('b2126cf8-d76f-4985-96b1-3aae11bfbd6b')])
)

[]

In [ ]:
# TODO maybe add batch_find_many_in?

In [28]:
from math_rag.application.models.assistants.inputs import (
    MathExpressionRelationshipDescriptionWriter as AssistantInput,
)
from math_rag.core.models import MathExpressionRelationshipDescription


math_article_chunk_ids = [
    math_expression_relationship.math_article_chunk_id
    for math_expression_relationship in math_expression_relationships
]
math_article_chunks = await math_article_chunk_repository.find_many(
    filter=dict(id=math_article_chunk_ids)  # TODO
)

inputs: list[AssistantInput] = []
input_id_to_math_expression_relationship_id: dict[UUID, UUID] = {}

for math_article_chunk, math_expression_relationship in zip(
    math_article_chunks, math_expression_relationships
):
    input = AssistantInput(
        chunk=math_article_chunk.text,
        source=math_expression_relationship.math_expression_source_index,
        target=math_expression_relationship.math_expression_target_index,
    )
    inputs.append(input)
    input_id_to_math_expression_relationship_id[input.id] = math_expression_relationship.id

outputs = await math_expression_relationship_description_writer_assistant.concurrent_assist(inputs)
descriptions = [
    MathExpressionRelationshipDescription(
        math_expression_index_id=index.id,
        math_expression_relationship_id=input_id_to_math_expression_relationship_id[
            output.input_id
        ],
        text=output.description,
    )
    for output in outputs
]
await math_expression_relationship_description_repository.insert_many(descriptions)

0


ValueError: Batch request 32c81965-fb9b-472a-a72b-b33471ece2f8 is empty